In [1]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

Using TensorFlow backend.


In [2]:
poems = np.load('dataset/normalized_poem_text_no_newline.npy')
raw_text = ' '.join(poems)
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))

In [3]:
test_text = raw_text[:150000] # taking a small subset to test on my weak machine 
n_chars = len(test_text)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)

Total Characters:  150000
Total Vocab:  39


In [4]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 100 # play with this number
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
    seq_in = test_text[i:i + seq_length]
    seq_out = test_text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])

n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)

Total Patterns:  149900


In [6]:
# reshape X to be [samples, time steps, features]
X = np.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

In [8]:
# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')

In [9]:
# define the checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [10]:
model.fit(X, y, epochs=50, batch_size=64, callbacks=callbacks_list)

Epoch 1/50
149900/149900 [==============================] - 1896s - loss: 2.8533  
Epoch 2/50
149900/149900 [==============================] - 1859s - loss: 2.6447  
Epoch 3/50
149900/149900 [==============================] - 1850s - loss: 2.5794  
Epoch 4/50
149900/149900 [==============================] - 1851s - loss: 2.5361  
Epoch 5/50
149900/149900 [==============================] - 1856s - loss: 2.5021  
Epoch 6/50
149900/149900 [==============================] - 1851s - loss: 2.4715  
Epoch 7/50
149900/149900 [==============================] - 1855s - loss: 2.4382  
Epoch 8/50
149900/149900 [==============================] - 1858s - loss: 2.4112  
Epoch 9/50
149900/149900 [==============================] - 1852s - loss: 2.3817  
Epoch 10/50
149900/149900 [==============================] - 1851s - loss: 2.3534  
Epoch 11/50
149900/149900 [==============================] - 1853s - loss: 2.3266  
Epoch 12/50
149900/149900 [==============================] - 1853s - loss: 2.2980  
E

# Generating Text

In [18]:
filename = "weights-improvement-49-1.8790.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [39]:
# pick a seed
seed = ' أهكذا أبدا تمضي أمانينا  نطوي الحياة وليل الموت يطوينا  تجري بنا سفن الأعمار ماخرة  بحر الوجود ولا ' # has to be 100 chars
pattern = [char_to_int[char] for char in seed]

print("Seed:")
print( "\"", seed, "\"\n")
# generate characters
for i in range(140):
    x = np.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x, verbose=0)
    index = np.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
print("\nDone.")

Seed:
"  أهكذا أبدا تمضي أمانينا  نطوي الحياة وليل الموت يطوينا  تجري بنا سفن الأعمار ماخرة  بحر الوجود ولا  "

تركر أم يحر ووا وحر النساء والذور والحياة وصيد من بنا أنا الذي أحب بينا على الساعة العارب أنا ما زلت عير ماء الأذياء والأهداء واذت تناقي مم 
Done.
